# Monthly Revenue Forecasting: Data Exploration & Requirements

**Business Objective:** Accurate monthly revenue forecasting for May (using data up to April)

**ML Task:** Time-aware supervised regression with forecasting constraints

---

## Notebook Objectives
1. Define the **ideal dataset** required for monthly revenue forecasting
2. Explore the **Online Retail.xlsx** dataset
3. Identify **data gaps** and their impact on forecasting quality
4. Map available data to forecasting requirements

---
## Part 1: Ideal Dataset Definition

### 1.1 Historical Monthly Revenue (CRITICAL)

| Attribute | Specification |
|-----------|---------------|
| **Description** | Aggregated monthly revenue figures |
| **Granularity** | Monthly totals |
| **Time Span** | Minimum 24-36 months for seasonality detection |
| **Why Needed** | Target variable; captures trends, seasonality, autoregressive patterns |
| **Temporal Relevance** | Must be complete up to April (current month) |

**Impact if Missing:** Cannot train any forecasting model. FATAL.

---

### 1.2 Daily/Weekly Sales Aggregated Monthly

| Attribute | Specification |
|-----------|---------------|
| **Description** | Transaction-level or daily sales data |
| **Granularity** | Daily transactions → aggregated to monthly |
| **Why Needed** | Enables within-month pattern analysis, volatility measurement |
| **Temporal Relevance** | Lag features from t-1, t-2, t-3 months |

**Impact if Missing:** Lose intra-month dynamics; forecast stability may suffer.

---

### 1.3 Marketing Spend by Channel

| Attribute | Specification |
|-----------|---------------|
| **Description** | Monthly marketing expenditure by channel |
| **Channels** | Digital, TV, Print, Email, Social |
| **Why Needed** | Marketing drives revenue with lag effect (H3 hypothesis) |
| **Temporal Relevance** | 1-2 month lag before revenue impact |

**Impact if Missing:** Cannot model marketing ROI or lead effects.

---

### 1.4 Promotions / Discounts

| Attribute | Specification |
|-----------|---------------|
| **Description** | Promotional events, discount periods, campaigns |
| **Granularity** | Event dates and discount percentages |
| **Why Needed** | Promotions create revenue spikes; must account for or model |
| **Temporal Relevance** | Concurrent month effect |

**Impact if Missing:** Unexplained variance in revenue; reduced accuracy during promotional periods.

---

### 1.5 Customer Counts / Transactions

| Attribute | Specification |
|-----------|---------------|
| **Description** | Unique customers, transaction counts, repeat rates |
| **Granularity** | Monthly aggregates |
| **Why Needed** | Revenue = Customers × Avg. Transaction Value; decomposition |
| **Temporal Relevance** | Leading indicator for revenue |

**Impact if Missing:** Cannot decompose revenue drivers; limited interpretability.

---

### 1.6 External Signals (Optional but Valuable)

| Signal | Description | Why Needed |
|--------|-------------|------------|
| **Holidays** | National/regional holidays | Explains demand spikes/dips |
| **Business Days** | Working days per month | Normalizes revenue per day |
| **Seasonality Indicators** | Quarter, fiscal period | Captures business cycles |
| **Economic Indicators** | CPI, consumer confidence | External demand drivers |

**Impact if Missing:** Model relies solely on internal data; may miss macro trends.

---
## Part 2: Load and Explore Online Retail Dataset

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

In [ ]:
# Load the Online Retail Dataset
DATA_PATH = '../data/raw/Online Retail.xlsx'

# Load data
df = pd.read_excel(DATA_PATH)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

In [ ]:
# Display first few rows and basic info
print("=" * 60)
print("DATASET PREVIEW")
print("=" * 60)
display(df.head(10))

In [ ]:
# Dataset Schema and Data Types
print("=" * 60)
print("COLUMN INFORMATION")
print("=" * 60)
print(f"\nColumns: {list(df.columns)}")
print(f"\nData Types:")
print(df.dtypes)
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Statistical Summary
print("=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
display(df.describe())

---
## Part 3: Data Quality Assessment

In [ ]:
# Missing Values Analysis
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing Count': df.isnull().sum().values,
    'Missing %': (df.isnull().sum().values / len(df) * 100).round(2),
    'Non-Null Count': df.notnull().sum().values,
    'Data Type': df.dtypes.values
})

missing_df = missing_df.sort_values('Missing %', ascending=False)
display(missing_df)

In [ ]:
# Visualize Missing Data
fig, ax = plt.subplots(figsize=(10, 5))

colors = ['#e74c3c' if x > 0 else '#27ae60' for x in missing_df['Missing %']]
bars = ax.barh(missing_df['Column'], missing_df['Missing %'], color=colors)

ax.set_xlabel('Missing Percentage (%)', fontsize=12)
ax.set_title('Missing Values by Column', fontsize=14, fontweight='bold')
ax.axvline(x=5, color='orange', linestyle='--', label='5% threshold')

for bar, pct in zip(bars, missing_df['Missing %']):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{pct:.1f}%', va='center', fontsize=10)

plt.legend()
plt.tight_layout()
plt.show()

---
## Part 4: Temporal Analysis

In [ ]:
# Parse and analyze date column
# Identify the date column (common names: InvoiceDate, Date, OrderDate)
date_columns = [col for col in df.columns if 'date' in col.lower()]
print(f"Detected date columns: {date_columns}")

if date_columns:
    date_col = date_columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    print(f"\n{'=' * 60}")
    print(f"TEMPORAL COVERAGE ANALYSIS")
    print(f"{'=' * 60}")
    print(f"Date Column: {date_col}")
    print(f"Date Range: {df[date_col].min()} to {df[date_col].max()}")
    print(f"Total Duration: {(df[date_col].max() - df[date_col].min()).days} days")
    print(f"Approximate Months: {(df[date_col].max() - df[date_col].min()).days // 30} months")
else:
    print("No date column found! Check column names.")

In [ ]:
# Extract temporal features
if date_columns:
    date_col = date_columns[0]
    
    df['Year'] = df[date_col].dt.year
    df['Month'] = df[date_col].dt.month
    df['YearMonth'] = df[date_col].dt.to_period('M')
    df['DayOfWeek'] = df[date_col].dt.dayofweek
    df['DayOfMonth'] = df[date_col].dt.day
    df['WeekOfYear'] = df[date_col].dt.isocalendar().week
    
    print("Temporal features extracted:")
    print(df[['Year', 'Month', 'YearMonth', 'DayOfWeek', 'DayOfMonth', 'WeekOfYear']].head())

In [ ]:
# Monthly transaction distribution
print("=" * 60)
print("TRANSACTIONS BY YEAR-MONTH")
print("=" * 60)

monthly_counts = df.groupby('YearMonth').size()
print(monthly_counts)

fig, ax = plt.subplots(figsize=(14, 5))
monthly_counts.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_xlabel('Year-Month', fontsize=12)
ax.set_ylabel('Transaction Count', fontsize=12)
ax.set_title('Transaction Volume by Month', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## Part 5: Revenue Calculation & Analysis

In [ ]:
# Identify revenue-related columns
print("=" * 60)
print("REVENUE COLUMN IDENTIFICATION")
print("=" * 60)

# Common patterns for revenue columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric columns: {numeric_cols}")

# Look for price and quantity columns
price_cols = [col for col in df.columns if any(x in col.lower() for x in ['price', 'amount', 'value'])]
qty_cols = [col for col in df.columns if any(x in col.lower() for x in ['quantity', 'qty', 'units'])]

print(f"\nPrice-related columns: {price_cols}")
print(f"Quantity-related columns: {qty_cols}")

In [ ]:
# Calculate Revenue (Quantity × UnitPrice) - Standard for Online Retail dataset
# Adjust column names based on actual dataset

# Try common column name patterns
qty_col = None
price_col = None

for col in df.columns:
    if 'quantity' in col.lower():
        qty_col = col
    if 'price' in col.lower() and 'unit' in col.lower():
        price_col = col
    elif 'price' in col.lower():
        price_col = col

print(f"Quantity column: {qty_col}")
print(f"Price column: {price_col}")

if qty_col and price_col:
    df['Revenue'] = df[qty_col] * df[price_col]
    print(f"\nRevenue column created: Quantity × UnitPrice")
    print(f"Revenue Statistics:")
    print(df['Revenue'].describe())
else:
    print("Could not identify quantity/price columns. Manual mapping required.")

In [ ]:
# Data Quality: Check for negative quantities and prices (returns/cancellations)
print("=" * 60)
print("DATA QUALITY: NEGATIVE VALUES ANALYSIS")
print("=" * 60)

if qty_col:
    neg_qty = (df[qty_col] < 0).sum()
    print(f"Negative quantities (returns): {neg_qty:,} ({neg_qty/len(df)*100:.2f}%)")

if price_col:
    neg_price = (df[price_col] <= 0).sum()
    print(f"Zero/Negative prices: {neg_price:,} ({neg_price/len(df)*100:.2f}%)")

if 'Revenue' in df.columns:
    neg_rev = (df['Revenue'] < 0).sum()
    zero_rev = (df['Revenue'] == 0).sum()
    print(f"Negative revenue: {neg_rev:,} ({neg_rev/len(df)*100:.2f}%)")
    print(f"Zero revenue: {zero_rev:,} ({zero_rev/len(df)*100:.2f}%)")

In [ ]:
# Filter for valid transactions (positive revenue for forecasting)
print("=" * 60)
print("FILTERING VALID TRANSACTIONS")
print("=" * 60)

if 'Revenue' in df.columns:
    df_valid = df[df['Revenue'] > 0].copy()
    print(f"Original records: {len(df):,}")
    print(f"Valid transactions (Revenue > 0): {len(df_valid):,}")
    print(f"Removed: {len(df) - len(df_valid):,} ({(len(df) - len(df_valid))/len(df)*100:.2f}%)")
else:
    df_valid = df.copy()

---
## Part 6: Monthly Revenue Aggregation

In [ ]:
# Aggregate revenue by month - THIS IS OUR TARGET VARIABLE
print("=" * 60)
print("MONTHLY REVENUE AGGREGATION")
print("=" * 60)

if 'Revenue' in df_valid.columns and 'YearMonth' in df_valid.columns:
    monthly_revenue = df_valid.groupby('YearMonth').agg({
        'Revenue': 'sum',
        qty_col: 'sum' if qty_col else 'count',
    }).reset_index()
    
    # Add transaction count
    monthly_txn_count = df_valid.groupby('YearMonth').size().reset_index(name='TransactionCount')
    monthly_revenue = monthly_revenue.merge(monthly_txn_count, on='YearMonth')
    
    # Add unique customer count if CustomerID exists
    customer_cols = [col for col in df_valid.columns if 'customer' in col.lower()]
    if customer_cols:
        customer_col = customer_cols[0]
        monthly_customers = df_valid.groupby('YearMonth')[customer_col].nunique().reset_index(name='UniqueCustomers')
        monthly_revenue = monthly_revenue.merge(monthly_customers, on='YearMonth')
    
    monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].astype(str)
    
    print("\nMonthly Revenue Summary:")
    display(monthly_revenue)
else:
    print("Cannot aggregate - Revenue or YearMonth column missing")

In [ ]:
# Visualize Monthly Revenue Trend
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Monthly Revenue
ax1 = axes[0, 0]
ax1.plot(monthly_revenue['YearMonth'], monthly_revenue['Revenue'], 
         marker='o', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('Month')
ax1.set_ylabel('Revenue')
ax1.set_title('Monthly Revenue Trend', fontweight='bold', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Plot 2: Transaction Count
ax2 = axes[0, 1]
ax2.bar(monthly_revenue['YearMonth'], monthly_revenue['TransactionCount'], 
        color='#3498db', edgecolor='black')
ax2.set_xlabel('Month')
ax2.set_ylabel('Transaction Count')
ax2.set_title('Monthly Transaction Volume', fontweight='bold', fontsize=12)
ax2.tick_params(axis='x', rotation=45)

# Plot 3: Unique Customers (if available)
ax3 = axes[1, 0]
if 'UniqueCustomers' in monthly_revenue.columns:
    ax3.plot(monthly_revenue['YearMonth'], monthly_revenue['UniqueCustomers'], 
             marker='s', linewidth=2, markersize=8, color='#9b59b6')
    ax3.set_ylabel('Unique Customers')
    ax3.set_title('Monthly Unique Customers', fontweight='bold', fontsize=12)
else:
    ax3.text(0.5, 0.5, 'Customer data not available', ha='center', va='center')
ax3.set_xlabel('Month')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# Plot 4: Average Revenue per Transaction
ax4 = axes[1, 1]
monthly_revenue['AvgRevPerTxn'] = monthly_revenue['Revenue'] / monthly_revenue['TransactionCount']
ax4.bar(monthly_revenue['YearMonth'], monthly_revenue['AvgRevPerTxn'], 
        color='#e74c3c', edgecolor='black')
ax4.set_xlabel('Month')
ax4.set_ylabel('Avg Revenue / Transaction')
ax4.set_title('Average Transaction Value by Month', fontweight='bold', fontsize=12)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## Part 7: Data Gap Analysis - Available vs. Ideal

In [ ]:
# Compare available data against ideal requirements
print("=" * 80)
print("DATA GAP ANALYSIS: AVAILABLE vs. IDEAL")
print("=" * 80)

data_requirements = {
    'Historical Monthly Revenue': {
        'Required': True,
        'Available': 'Revenue' in df.columns,
        'Source': 'Calculated from Quantity × UnitPrice',
        'Impact if Missing': 'FATAL - Cannot train any model'
    },
    'Transaction-level Data': {
        'Required': True,
        'Available': True,
        'Source': 'Each row is a transaction line item',
        'Impact if Missing': 'Lose intra-month patterns'
    },
    'Customer Information': {
        'Required': True,
        'Available': any('customer' in col.lower() for col in df.columns),
        'Source': 'CustomerID column',
        'Impact if Missing': 'Cannot analyze customer behavior'
    },
    'Product Information': {
        'Required': False,
        'Available': any('stock' in col.lower() or 'product' in col.lower() or 'description' in col.lower() for col in df.columns),
        'Source': 'StockCode/Description columns',
        'Impact if Missing': 'Cannot do product-level analysis'
    },
    'Geographic Data': {
        'Required': False,
        'Available': any('country' in col.lower() for col in df.columns),
        'Source': 'Country column',
        'Impact if Missing': 'Cannot analyze regional trends'
    },
    'Marketing Spend': {
        'Required': True,
        'Available': False,
        'Source': 'NOT IN DATASET',
        'Impact if Missing': 'Cannot test H3 (marketing lead effect)'
    },
    'Promotions/Discounts': {
        'Required': True,
        'Available': False,
        'Source': 'NOT IN DATASET',
        'Impact if Missing': 'Cannot explain promotional spikes'
    },
    'External Economic Indicators': {
        'Required': False,
        'Available': False,
        'Source': 'NOT IN DATASET (can be added externally)',
        'Impact if Missing': 'Limited macro-economic context'
    },
    'Holiday Calendar': {
        'Required': False,
        'Available': False,
        'Source': 'NOT IN DATASET (can be engineered)',
        'Impact if Missing': 'May miss holiday effects'
    }
}

gap_df = pd.DataFrame(data_requirements).T
gap_df.index.name = 'Data Requirement'

# Color coding
def highlight_availability(val):
    if val == True:
        return 'background-color: #27ae60; color: white'
    elif val == False:
        return 'background-color: #e74c3c; color: white'
    return ''

display(gap_df.style.applymap(highlight_availability, subset=['Available', 'Required']))

In [ ]:
# Summary Statistics for Forecasting Readiness
print("=" * 80)
print("FORECASTING READINESS SUMMARY")
print("=" * 80)

available_count = sum(1 for v in data_requirements.values() if v['Available'])
total_count = len(data_requirements)
required_available = sum(1 for v in data_requirements.values() if v['Required'] and v['Available'])
required_total = sum(1 for v in data_requirements.values() if v['Required'])

print(f"\n📊 Data Availability: {available_count}/{total_count} ({available_count/total_count*100:.0f}%)")
print(f"🎯 Required Data Available: {required_available}/{required_total} ({required_available/required_total*100:.0f}%)")

print("\n" + "=" * 80)
print("HYPOTHESIS TESTABILITY ASSESSMENT")
print("=" * 80)

hypotheses = {
    'H1: Autoregressive Revenue': {'Testable': True, 'Reason': 'Have historical revenue data'},
    'H2: Seasonality Patterns': {'Testable': True, 'Reason': 'Have date/time information'},
    'H3: Marketing Lead Effect': {'Testable': False, 'Reason': 'No marketing spend data'},
    'H4: Economic Indicators': {'Testable': False, 'Reason': 'No external economic data'},
    'H5: Business Day Effect': {'Testable': True, 'Reason': 'Can calculate from dates'},
}

for h, info in hypotheses.items():
    status = '✅' if info['Testable'] else '❌'
    print(f"{status} {h}: {info['Reason']}")

---
## Part 8: Key Findings Summary

In [ ]:
# Generate comprehensive data profile
print("=" * 80)
print("📋 DATASET PROFILE SUMMARY")
print("=" * 80)

profile = {
    'Total Records': f"{len(df):,}",
    'Valid Transactions': f"{len(df_valid):,}" if 'df_valid' in dir() else 'N/A',
    'Columns': len(df.columns),
    'Date Range': f"{df[date_col].min().date()} to {df[date_col].max().date()}" if date_columns else 'N/A',
    'Months Covered': len(monthly_revenue) if 'monthly_revenue' in dir() else 'N/A',
    'Total Revenue': f"${monthly_revenue['Revenue'].sum():,.2f}" if 'monthly_revenue' in dir() else 'N/A',
    'Avg Monthly Revenue': f"${monthly_revenue['Revenue'].mean():,.2f}" if 'monthly_revenue' in dir() else 'N/A',
    'Unique Customers': f"{df_valid[customer_col].nunique():,}" if 'customer_col' in dir() and customer_col else 'N/A',
}

for key, value in profile.items():
    print(f"  {key}: {value}")

print("\n" + "=" * 80)
print("🚦 FORECASTING FEASIBILITY")
print("=" * 80)
print("""
✅ STRENGTHS:
   • Transaction-level granularity enables rich feature engineering
   • Customer ID allows customer behavior analysis
   • Product/Stock data supports category-level forecasting
   • Geographic data enables regional trend analysis

⚠️ LIMITATIONS:
   • No marketing spend data - cannot test marketing ROI hypothesis
   • No promotional calendar - promotional effects unexplained
   • No external economic indicators - limited macro context
   • Time span may be limited - check if 24+ months available

🎯 RECOMMENDED APPROACH:
   1. Focus on autoregressive and seasonal models (H1, H2)
   2. Engineer business day features (H5)
   3. Create customer/product-based features
   4. Consider external data augmentation if needed
""")

In [ ]:
# Save processed monthly data for modeling
if 'monthly_revenue' in dir():
    output_path = '../data/processed/monthly_revenue.csv'
    monthly_revenue.to_csv(output_path, index=False)
    print(f"✅ Monthly revenue data saved to: {output_path}")
    print(f"   Records: {len(monthly_revenue)} months")
    print(f"   Columns: {list(monthly_revenue.columns)}")

---
## Next Steps

1. **Data Preprocessing** → Clean and transform data for modeling
2. **Feature Engineering** → Create lag features, seasonal indicators, business day counts
3. **Hypothesis Testing** → Validate H1, H2, H5 using statistical tests
4. **Model Development** → Build baseline and ML forecasting models
5. **Evaluation** → Assess forecast accuracy and stability

---
*Notebook created for Monthly Revenue Forecasting Project*